In [1]:
import glob
import numpy as np
import xarray as xr

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
test_files = glob.glob('/home/hanna/lagrings/ERA5_monthly/*2012*01*.nc')
data = xr.open_mfdataset(test_files, compat='no_conflicts')

/home/hanna/anaconda3/envs/final/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  
/home/hanna/anaconda3/envs/final/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).The datasets supplied require both concatenation and merging. From
xarray version 0.15 this will operation will requir

In [4]:
subset = data.sel(latitude = 32, longitude = 0.0)

In [5]:
from timeit import default_timer as timer

from sclouds.ml.regression.utils import (mean_squared_error, r2_score, fit_pixel,
                                         predict_pixel,
                                         accumulated_squared_error,
                                         sigmoid, inverse_sigmoid)

from sclouds.ml.regression.utils import (dataset_to_numpy, dataset_to_numpy_order,
                                         dataset_to_numpy_order_traditional_ar,
                                         dataset_to_numpy_grid_order,
                                         dataset_to_numpy_grid,
                                         get_xarray_dataset_for_period,
                                         get_list_of_files_excluding_period,
                                         get_list_of_files,
                                         get_list_of_files_excluding_period_traditional_model,
                                         get_list_of_files_traditional_model)

#sys.path.insert(0,'/uio/hume/student-u89/hannasv/MS/sclouds/')
from sclouds.helpers import (merge, get_list_of_variables_in_ds,
                             get_pixel_from_ds, path_input, path_ar_results)

Warning.. Using pgf backend, no GUI available. use plt.savefig() for inpection


In [27]:
bias = False 
order = 4
type = 'ar'

def get_X_and_y(dataset):
    """Returns X and y based on a dataset. """
    if type == 'ar':
        if order > 0:
            X, y = dataset_to_numpy_order(dataset, order, bias = bias)
        else:
            X, y  = dataset_to_numpy(dataset, bias = bias)
    else:
        X, y = dataset_to_numpy_order_traditional_ar(
                                dataset, order = order, bias = bias)
    return X, y

In [28]:
X, y = get_X_and_y(subset)

In [29]:
X.shape, y.shape

((740, 8), (740, 1))

# Models

In [30]:
coeff = fit_pixel(X, y)
coeff

array([[-1.06385409e+00],
       [ 5.63842080e-03],
       [ 1.71302057e-03],
       [-1.78400538e-05],
       [ 7.78543894e-01],
       [ 2.12529080e-02],
       [ 4.33246411e-02],
       [ 5.14919505e-02]])

# bias plus env variables

In [31]:
coeff = fit_pixel(X[:, :-1], y)
coeff

array([[-2.61769794e+00],
       [ 5.88383638e-03],
       [ 1.81425707e-03],
       [-1.85719874e-05],
       [ 7.82849196e-01],
       [ 2.24803970e-02],
       [ 8.42678577e-02]])

## Only enviornmental variables. 

In [32]:
coeff = fit_pixel(X[:, 1:-1], y)
coeff

array([[ 5.55005623e-03],
       [ 1.68034962e-03],
       [-1.75866669e-05],
       [ 7.83026789e-01],
       [ 2.24940122e-02],
       [ 8.47918468e-02]])

# Traditional AR-model 

In [33]:
coeff = fit_pixel(X[:, -1, np.newaxis], y)
coeff

array([[0.79297021]])

# Problem with this is that is doesnt include different amounts of samples for different numbers of order.
## Need to remove rows with nans after removing sumbset.